In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
from os import path
import tempfile
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.options.mode.chained_assignment = None

import seaborn as sns

import sklearn

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

### DataFrame name:

In [ ]:
df_name = "95"
layer1nodes = 128
suffix = ""

df_path = f"/content/drive/MyDrive/uni/RoboDoc/ml/{df_name}"

model_name = f"df{df_name}_n{layer1nodes}{suffix}"

model_path = f"{df_path}/{model_name}"

if path.exists(model_path) == False:
  os.mkdir(model_path)

In [ ]:
df = pd.read_pickle(f"{df_path}/dataframe_{df_name}.pkl")

In [ ]:
df.head()

,age,gender,weight,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,length_of_stay_hospital,died_in_hospital,days_to_death
0,0.578947,1.0,0.317726,0.510949,0.448,0.463768,0.182243,0.311475,0.205128,0.471545,0.132229,0.081716,0.070480,0.364043,0.456363,0.553465,0.421632,0.460921,0.230861,0.364046,0.150707,0.616188,0.387668,0.408317,0.490798,0.149982,0.348723,0.410745,0.410293,0.375371,0.294447,0.420647,0.483331,0.459979,0.507799,0.475832,0.325318,0.623990,0.463194,0.489838,...,0.451496,0.469973,0.400281,0.444384,0.462615,0.452883,0.471940,0.483980,0.506908,0.452392,0.423360,0.487804,0.445244,0.450775,0.454335,0.481367,0.511838,0.448053,0.476317,0.403012,0.478884,0.382877,0.484026,0.464456,0.431639,0.490876,0.452463,0.459690,0.392050,0.466125,0.426711,0.454286,0.472632,0.478366,0.446211,0.394671,0.496392,10.61,0,5.0
1,0.526316,1.0,0.266388,0.562044,0.552,0.326087,0.275701,0.311475,0.358974,0.621500,0.238798,0.136874,0.127681,0.273684,0.376123,0.547164,0.110738,0.426807,0.536119,0.491203,0.391604,0.530948,0.233122,0.285013,0.545109,0.354498,0.474979,0.550267,0.334811,0.389495,0.428739,0.409122,0.567919,0.344699,0.625053,0.610797,0.553657,0.749607,0.511606,0.530965,...,0.461610,0.449144,0.512688,0.538255,0.435267,0.407335,0.325974,0.536346,0.589737,0.381024,0.420131,0.466249,0.457916,0.385405,0.433735,0.485510,0.500216,0.520159,0.308390,0.373872,0.574494,0.369746,0.538771,0.360851,0.739281,0.507123,0.537671,0.578029,0.341291,0.479565,0.496294,0.530438,0.448549,0.252540,0.324613,0.363326,0.554716,60.01,0,28.0
2,0.750000,1.0,0.301003,0.430657,0.520,0.264493,0.252336,0.311475,0.282051,0.579301,0.172827,0.158325,0.121553,0.395083,0.169589,0.334067,0.220507,0.569524,0.656565,0.469736,0.432011,0.273094,0.391768,0.399913,0.451684,0.371500,0.413204,0.452774,0.506542,0.468486,0.377149,0.424454,0.546171,0.337927,0.519142,0.433113,0.459250,0.547283,0.418791,0.246391,...,0.449899,0.485622,0.389515,0.429574,0.429249,0.471155,0.437163,0.490338,0.481424,0.457246,0.418201,0.462029,0.441233,0.449240,0.491772,0.454483,0.463925,0.461486,0.504925,0.400865,0.494406,0.378507,0.477134,0.432877,0.472330,0.488397,0.439208,0.479703,0.392930,0.463207,0.435096,0.468214,0.502176,0.458563,0.427514,0.433890,0.472819,6.33,0,NaN
3,0.723684,1.0,0.287625,0.459854,0.448,0.311594,0.135514,0.327869,0.282051,0.661340,0.177901,0.136874,0.111338,0.526008,0.478596,0.180866,0.254238,0.537086,0.531832,0.402759,0.404065,0.667459,0.410731,0.404282,0.536918,0.121502,0.333665,0.521657,0.404536,0.423838,0.230396,0.438998,0.398284,0.421840,0.543324,0.451551,0.291949,0.525944,0.445496,0.616157,...,0.596357,0.217947,0.280199,0.272401,0.668163,0.296712,0.558334,0.420042,0.856998,0.212607,0.282565,0.785182,0.540643,0.365364,0.432607,0.569005,0.495348,0.799436,0.449475,0.424703,0.472156,0.122250,0.406786,0.473348,0.339143,0.527691,0.533946,0.266586,0.357349,0.398143,0.370468,0.660747,0.717472,0.474361,0.420944,0.505622,0.386065,6.02,0,NaN
4,0.394737,0.0,0.266388,0.394161,0.400,0.221014,0.369159,0.262295,0.307692,0.558266,0.220190,0.165475,0.131767,0.042262,0.416218,0.284472,0.368381,0.484330,0.438105,0.469459,0.463685,0.314813,0.417263,0.441096,0.432818,0.408774,0.364042,0.458567,0.413726,0.471160,0.389235,0.456072,0.467205,0.485622,0.483965,0.394747,0.438353,0.481559,0.474867,0.499240,...,0.448158,0.495605,0.405968,0.455817,0.454258,0.474887,0.451451,0.502501,0.488990,0.505782,0.402557,0.451956,0.446834,0.456246,0.468295,0.462567,0.472640,0.474050,0.516415,0.400370,0.477945,0.391255,0.456062,0.430766,0.487166,0.483881,0.461030,0.494312,0.403677,0.473061,0.455825,0.497913,0.516634,0.459559,0.400484,0.416300,0.465035,2.01,0,NaN


In [ ]:
df.describe()

,age,gender,weight,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,length_of_stay_hospital,days_to_death
count,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,...,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,21444.000000
mean,0.650590,0.562129,0.266388,0.445591,0.467871,0.296353,0.283202,0.313857,0.301810,0.608499,0.180799,0.162328,0.120644,0.262751,0.404717,0.369815,0.386130,0.490819,0.443567,0.466785,0.455792,0.338298,0.435870,0.436951,0.453190,0.424337,0.405111,0.463086,0.427688,0.462124,0.394588,0.462222,0.465889,0.468876,0.499696,0.397292,0.443534,0.482537,0.492243,0.495614,...,0.493126,0.455065,0.485833,0.393557,0.442577,0.434469,0.455794,0.446471,0.488138,0.476483,0.459526,0.418764,0.458832,0.430521,0.447943,0.475411,0.464631,0.472509,0.451623,0.493592,0.408311,0.494183,0.380829,0.483436,0.434358,0.469886,0.486177,0.448833,0.485848,0.402913,0.468509,0.443845,0.470479,0.494462,0.457499,0.425124,0.426741,0.480849,9.990334,12.628474
std,0.228188,0.496130,0.072670,0.082701,0.109438,0.094120,0.086976,0.106607,0.102414,0.060794,0.058942,0.087584,0.045213,0.177362,0.173491,0.128753,0.133040,0.122872,0.116459,0.130797,0.117562,0.119592,0.122049,0.100812,0.108233,0.109583,0.093970,0.099768,0.124292,0.123155,0.125566,0.104172,0.093376,0.095048,0.094871,0.091836,0.093674,0.087280,0.100602,0.078760,...,0.042387,0.041216,0.040174,0.034711,0.044265,0.042217,0.034033,0.037864,0.036830,0.044027,0.046013,0.030993,0.039151,0.036317,0.037540,0.047838,0.031877,0.038163,0.039022,0.037780,0.029666,0.045057,0.033834,0.039320,0.039902,0.044196,0.033762,0.043657,0.045517,0.034513,0.033318,0.036002,0.039627,0.045321,0.041871,0.035925,0.037507,0.045871,10.812359,9.036426
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000
25%,0.500000,0.000000,0.220736,0.386861,0.408000,0.242754,0.228972,0.245902,0.230769,0.571816,0.145762,0.107252,0.091931,0.113990,0.265941,0.280532,0.324501,0.430862,0.377935,0.387450,0.398253,0.268717,0.371063,0.388484,0.415418,0.364886,0.354959,0.407442,0.366554,0.395420,0.331553,0.422746,0.408808,0.412807,0.463294

## Examine the class label imbalance:

In [ ]:
neg, pos = np.bincount(df['died_in_hospital'])
total = neg + pos
print(f'Examples:\n    Total: {total}\n    Positive: {pos} ({100 * pos / total:.2f}% of total)\n')

Examples:
    Total: 48866
    Positive: 5612 (11.48% of total)



### Pop unnecessary labels:

In [ ]:
df.pop("days_to_death")
df.pop("length_of_stay_hospital")

0        10.61
1        60.01
2         6.33
3         6.02
4         2.01
         ...  
48861    15.87
48862     7.86
48863    13.24
48864     3.14
48865     2.82
Name: length_of_stay_hospital, Length: 48866, dtype: float64

## Split data:

In [ ]:
print(df.columns[:20])
print(df.columns[-5:])

train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(df, test_size=0.2)

train_labels = np.array(train_df.pop('died_in_hospital')).astype('float32')
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('died_in_hospital')).astype('float32')
test_labels = np.array(test_df.pop('died_in_hospital')).astype('float32')

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

Index([          'age',        'gender',        'weight',   'meanbp_mean',
          'meanbp_min',    'meanbp_max',  'resprate_min',  'resprate_max',
       'resprate_mean',    'tempc_mean',   'glucose_min',   'glucose_max',
        'glucose_mean',               0,               1,               2,
                     3,               4,               5,               6],
      dtype='object')
Index([1432, 1433, 1434, 1435, 'died_in_hospital'], dtype='object')


In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)



Training labels shape: (39092,)
Validation labels shape: (9774,)
Test labels shape: (9774,)
Training features shape: (39092, 1449)
Validation features shape: (9774, 1449)
Test features shape: (9774, 1449)


## Define the model and metrics:

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential([
        keras.layers.Dense(layer1nodes, activation='relu', input_shape=(train_features.shape[-1],)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)

    return model

In [ ]:
EPOCHS = 512
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=24,
    mode='max',
    restore_best_weights=True)

In [ ]:
initial_bias = np.log([pos/neg])

model = make_model(output_bias=initial_bias)
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 448)               649600    
_________________________________________________________________
dropout_18 (Dropout)         (None, 448)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 449       
Total params: 650,049
Trainable params: 650,049
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.predict(train_features[-10:])

array([[0.12513283],
       [0.07756627],
       [0.11195478],
       [0.02580878],
       [0.20908916],
       [0.331079  ],
       [0.17120913],
       [0.24019024],
       [0.48413056],
       [0.10763434]], dtype=float32)

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 0.4107


### Save initial Biases for better comparison:

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')

# Train model:

In [ ]:
"""
train_features_tensor = tf.convert_to_tensor(train_features, dtype=tf.int64)
train_labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.int64)
val_features_tensor = tf.convert_to_tensor(val_features, dtype=tf.int64)
val_labels_tensor = tf.convert_to_tensor(val_labels, dtype=tf.int64)
test_features_tensor = tf.convert_to_tensor(test_features, dtype=tf.int64)
test_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.int64)
"""

'\ntrain_features_tensor = tf.convert_to_tensor(train_features, dtype=tf.int64)\ntrain_labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.int64)\nval_features_tensor = tf.convert_to_tensor(val_features, dtype=tf.int64)\nval_labels_tensor = tf.convert_to_tensor(val_labels, dtype=tf.int64)\ntest_features_tensor = tf.convert_to_tensor(test_features, dtype=tf.int64)\ntest_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.int64)\n'

## Weighted Classes:

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.56
Weight for class 1: 4.35


In [ ]:
weighted_model = make_model()

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/512
20/20 [==============================] - 6s 180ms/step - loss: 0.7102 - tp: 3003.0000 - fp: 11536.0000 - tn: 57682.0000 - fn: 5963.0000 - accuracy: 0.7762 - precision: 0.2065 - recall: 0.3349 - auc: 0.5872 - prc: 0.2115 - val_loss: 0.5273 - val_tp: 951.0000 - val_fp: 2081.0000 - val_tn: 6564.0000 - val_fn: 178.0000 - val_accuracy: 0.7689 - val_precision: 0.3137 - val_recall: 0.8423 - val_auc: 0.8775 - val_prc: 0.5555
Epoch 2/512
20/20 [==============================] - 2s 118ms/step - loss: 0.4082 - tp: 3689.0000 - fp: 6002.0000 - tn: 28607.0000 - fn: 794.0000 - accuracy: 0.8262 - precision: 0.3807 - recall: 0.8229 - auc: 0.9000 - prc: 0.6053 - val_loss: 0.3610 - val_tp: 854.0000 - val_fp: 1136.0000 - val_tn: 7509.0000 - val_fn: 275.0000 - val_accuracy: 0.8556 - val_precision: 0.4291 - val_recall: 0.7564 - val_auc: 0.8898 - val_prc: 0.6006
Epoch 3/512
20/20 [==============================] - 2s 118ms/step - loss: 0.3265 - tp: 3901.0000 - fp: 4980.0000 - tn: 29629.0000 - fn:

## Resampled data:

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

neg_features.shape

(34609, 1449)

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
    ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
    ds = ds.shuffle(BUFFER_SIZE).repeat()
    return ds

"""
pos_features_tensor = tf.convert_to_tensor(pos_features, dtype=tf.int64)
pos_labels_tensor = tf.convert_to_tensor(pos_labels, dtype=tf.int64)

neg_features_tensor = tf.convert_to_tensor(neg_features, dtype=tf.int64)
neg_labels_tensor = tf.convert_to_tensor(neg_labels, dtype=tf.int64)
"""

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)


In [ ]:
for features, label in resampled_ds.take(1):
    print(label.numpy().mean())

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

In [ ]:
resampled_model = make_model()

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs (added to avoid overfitting)
    steps_per_epoch=20,
    epochs=10*EPOCHS,
    callbacks=[early_stopping],
    validation_data=val_ds)

# Evaluate training history:

- Accuracy is the percentage of examples correctly classified
- Precision is the percentage of predicted positives that were correctly classified
- Recall is the percentage of actual positives that were correctly classified
- AUC refers to the Area Under the Curve of a Receiver Operating Characteristic curve (ROC-AUC). This metric is equal to the probability that a classifier will rank a random positive sample higher than a random negative sample.
- AUPRC refers to Area Under the Curve of the Precision-Recall Curve. This metric computes precision-recall pairs for different probability thresholds.

In [ ]:
def plot_metrics(history, label, color):
    metrics = ['loss', 'prc', 'precision', 'recall']
    # metrics = ['tp', 'fp', 'tn', 'fn']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[color], label=label + ' Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[color], linestyle="--", label=label + ' Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        elif metric in ['tp', 'fp', 'fn']:
            plt.ylim([0, pos])
        elif metric in ['tn']:
            plt.ylim([0, neg])
        else:
            plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_metrics(weighted_history, "weighted", 1)
plot_metrics(resampled_history, "resampled", 2)

plt.savefig(f"{model_path}/metrics_{model_name}.png")

## Evaluate classification:

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

In [ ]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

plt.savefig(f"{model_path}/weighted_classification_{model_name}.png")

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)
plt.savefig(f"{model_path}/resampled_classification_{model_name}.png")

## Plot the ROC (receiver operating characteristic) Curve:

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5,25])
    plt.ylim([75,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_roc("Train Resampled", train_labels, train_predictions_resampled, color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled, color=colors[2], linestyle='--')

plt.legend(loc='upper left')
plt.savefig(f"{model_path}/roc_{model_name}.png")

## Plot the PRC (precision-recall curve)

In [ ]:
def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
plot_prc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_prc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_prc("Train Resampled", train_labels, train_predictions_resampled, color=colors[2])
plot_prc("Test Resampled", test_labels, test_predictions_resampled, color=colors[2], linestyle='--')

plt.legend(loc='upper right')
plt.savefig(f"{model_path}/prc_{model_name}.png")

In [ ]:
weighted_model.save(f"{model_path}/weighted_model_{model_name}")

In [ ]:
resampled_model.save(f"{model_path}/resampled_model_{model_name}")